In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

# -----------------------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------------------
data = pd.read_csv('../Data_Engineer_Part/13_Merge_Table/TABLE1.csv')  # ใส่ชื่อไฟล์จริง
# คาดว่ามีคอลัมน์: Eid, Agency_Name, Subject_Field, Publish_month, Year, Country, Title, Index_Terms

# -----------------------------------------------------------------------
# 2. Data Cleaning / Handling duplicates if needed
# -----------------------------------------------------------------------
# ลบ duplicates หากมี
data = data.drop_duplicates()

# ตรวจสอบ Missing Values ใน Index_Terms
data['Index_Terms'] = data['Index_Terms'].fillna("").astype(str)

# -----------------------------------------------------------------------
# 3. Split data by Eid to avoid data leakage
# -----------------------------------------------------------------------
unique_eids = data['Eid'].unique()
train_eids, test_eids = train_test_split(unique_eids, test_size=0.2, random_state=42)

train_df = data[data['Eid'].isin(train_eids)].reset_index(drop=True)
test_df = data[data['Eid'].isin(test_eids)].reset_index(drop=True)

# -----------------------------------------------------------------------
# 4. Encode Target (Agency_Name)
# -----------------------------------------------------------------------
le_agency = LabelEncoder()
train_df['Agency_label'] = le_agency.fit_transform(train_df['Agency_Name'])

# สำหรับ test set ถ้ามี Agency_Name ที่ไม่เคยเจอใน train ให้ลบออก
test_df = test_df[test_df['Agency_Name'].isin(le_agency.classes_)]
test_df['Agency_label'] = le_agency.transform(test_df['Agency_Name'])

# -----------------------------------------------------------------------
# 5. Prepare Features
# รวม Title กับ Index_Terms เข้าด้วยกันเป็นฟีเจอร์ข้อความเดียว
train_df['text_all'] = train_df['Title'].fillna("") + " " + train_df['Index_Terms'].fillna("")
test_df['text_all'] = test_df['Title'].fillna("") + " " + test_df['Index_Terms'].fillna("")

X_train = train_df[['text_all', 'Subject_Field', 'Country']]
y_train = train_df['Agency_label']

X_test = test_df[['text_all', 'Subject_Field', 'Country']]
y_test = test_df['Agency_label']

# -----------------------------------------------------------------------
# 6. Column Specification
# text_all จะใช้ TF-IDF
# Subject_Field, Country เป็นหมวดหมู่ (Categorical)
# Publish_month, Year เป็นตัวเลข (Numeric)
text_feature = 'text_all'
cat_features = ['Subject_Field', 'Country']

# -----------------------------------------------------------------------
# 7. Create Preprocessing Pipeline
# -----------------------------------------------------------------------
text_transformer = TfidfVectorizer(stop_words='english', max_features=1000)
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_feature),
        ('cat', cat_transformer, cat_features),
    ],
    remainder='drop'
)

# -----------------------------------------------------------------------
# 8. Handle Class Imbalance (If needed)
# -----------------------------------------------------------------------
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {c: w for c, w in zip(classes, class_weights)}

# -----------------------------------------------------------------------
# 9. Build Pipeline with Model
# -----------------------------------------------------------------------
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(n_jobs=-1, n_estimators=50, random_state=42, class_weight=class_weight_dict))
])

# -----------------------------------------------------------------------
# 10. Train Model
# -----------------------------------------------------------------------
model.fit(X_train, y_train)




Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(max_features=1000,
                                                                  stop_words='english'),
                                                  'text_all'),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Subject_Field',
                                                   'Country'])])),
                ('clf',
                 RandomForestClassifier(class_weight={0: 2.9843837367201855,
                                                      1: 17.90630242032111,
                                                      2: 0.6887039392431197,
                                                      3: 17.90630242032111,
                                                      4: 4.4765...
                                                      15: 0.08065901991135636,
                                                      16: 0.051902325856003226,
                                                      17: 17.90630242032111,
                                                      18: 8.953151210160556,
                                                      19: 17.90630242032111,
                                                      20: 8.953151210160556,
                                                      21: 8.953151210160556,
                                                      22: 17.90630242032111,
                                                      23: 17.90630242032111,
                                                      24: 8.953151210160556,
                                                      25: 4.476575605080278,
                                                      26: 8.953151210160556,
                                                      27: 4.476575605080278,
                                                      28: 2.238287802540139,
                                                      29: 0.8953151210160556, ...},
                                        n_estimators=50, n_jobs=-1,
                                        random_state=42))])

In [3]:
# -----------------------------------------------------------------------
# 11. Predict & Evaluate
# -----------------------------------------------------------------------


# -----------------------------------------------------------------------
# หมายเหตุ:
# - การมี Country ซ้ำไม่ถือว่าเป็น Data Leakage เนื่องจากเป็นคุณลักษณะตามธรรมชาติของข้อมูล:
#   หลาย Agency อาจมาจากประเทศเดียวกัน เป็นข้อมูลที่มีใน train ก็อาจมีใน test ได้ ไม่ได้เปิดเผยอะไรที่ทำให้
#   โมเดลเห็นข้อมูลเฉพาะของ test ล่วงหน้า
# - การป้องกัน Data Leakage สำคัญคือต้องไม่ให้ Eid เดียวกันปรากฏในทั้ง train และ test
# - หากต้องการปรับปรุงความแม่นยำ อาจลองปรับพารามิเตอร์ TF-IDF, โมเดล หรือทำ Feature Engineering เพิ่มเติม

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           2       0.00      0.00      0.00         9
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          13       0.18      0.06      0.09        87
          15       0.06      0.10      0.07        50
          16       0.53      0.75      0.62        77
          22       0.00      0.00      0.00         2
          27       1.00      1.00      1.00         2
          28       0.00      0.00      0.00         0
          29       0.00      0.00      0.00         6
          30       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         4
          43       0.00      0.00      0.00         0
          45       0.00      0.00      0.00        18
          51       0.00      0.00      0.00         2
          52       0.00      0.00      0.00         9
          60       1.00    

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/s